In [1]:
import pandas as pd

PREFIX = 'sample-ancestry-assignment.'
METADATA = '../data/metadata/metadata.tm.txt'
ANCESTRY = '../data/ancestry/ancestries.txt'

metadata = pd.read_csv(METADATA, sep='\t', index_col=None)
ancestry = pd.read_csv(ANCESTRY, sep='\t', index_col=0)
ancestry.head()

,AFR,SAS,EAS,EUR,AMR,OCN,MES
NWD_ID,,,,,,,
HG00096,0.0000,0.0014,0.0000,0.9982,0.0004,0.0,0.0000
HG00097,0.0000,0.0023,0.0000,0.9977,0.0000,0.0,0.0000
HG00099,0.0005,0.0036,0.0000,0.9937,0.0006,0.0,0.0015
HG00100,0.0002,0.0045,0.0000,0.9935,0.0004,0.0,0.0014
HG00101,0.0009,0.0000,0.0005,0.9979,0.0000,0.0,0.0008


In [2]:
metadata = metadata[metadata.used_for_scan]
sample_ancestries = ancestry.loc[metadata.wgs.unique(),:]
sample_ancestries['assignment'] = sample_ancestries.apply(lambda x: x.idxmax() if ((x.idxmax() == 'AMR' and x.max() >= 0.50) or (x.max() >= 0.75)) else 'unassigned', axis=1)
wgs_to_ancestry = dict(zip(sample_ancestries.index, sample_ancestries.assignment))


In [3]:
eqtl_samples = metadata.copy()
eqtl_samples['ancestry_assignment'] = eqtl_samples.wgs.map(wgs_to_ancestry)

In [4]:
eqtl_samples.head()

,cohort,sequencing_center,tor,tissue,inferred_sex,wgs,used_for_scan,ancestry_assignment
0,MESA,uw,TOR379757,PBMC,male,NWD331470,True,AMR
1,MESA,uw,TOR241007,Monocyte,male,NWD331470,True,AMR
2,MESA,uw,TOR461526,T_cell,male,NWD331470,True,AMR
3,MESA,uw,TOR882863,PBMC,male,NWD133576,True,EUR
5,MESA,broad,TOR811735,PBMC,female,NWD302101,True,EUR


In [5]:
eqtl_samples[['tissue', 'tor', 'wgs', 'ancestry_assignment']].to_csv(f'tables/{PREFIX}tsv', sep='\t', index=False)

In [6]:
ancestry_analysis_sample_sizes = eqtl_samples.groupby(['tissue', 'ancestry_assignment']).size().rename('n').reset_index()
ancestry_analysis_sample_sizes.to_csv(f'tables/{PREFIX}counts-per-tissue-per-ancestry.csv', index=False)